#**Práctica 8: Transferencia de aprendizaje (Transfer Learning)**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

## **INTRODUCCIÓN**

En la práctica, entrenar una red neuronal convolucional (ConvNet) diseñándola desde cero es raro, dado que pocas veces se cuenta con conjuntos de datos lo suficientemente grandes, es común pre-entrenar una ConvNet en un dataset muy grande (por ej. ImageNet, que contiene millones de imágenes con 1000 categorías), y luego usar esa ConvNet para la tarea de interés. Como se muestra en la siguiente figura ([fuente](https://www.mdpi.com/1424-8220/20/23/6713/htm) ), podemos aprovechar de dos formas principales estos modelos pre-entrenados (**a**), usándolos como un extractor de características (**b**) o como un modelo base al cual se le pueden ajustar algunas capas para mejorar los resultados de nuestro problema (**c**).

<center>
    <img width="60%" src="https://www.mdpi.com/sensors/sensors-20-06713/article_deploy/html/images/sensors-20-06713-g009.png">
</center>


En esta práctica veremos la estrategia de transferencia de aprendizaje (transfer learning) donde usaremos un modelo pre-entrenado en ImageNet para usarlo como base en un problema de clasificación binaria de avellanas en buen o mal estado en una linea de producción, las imagenes fueron descargadas y adaptadas desde [aquí](https://www.mvtec.com/company/research/datasets/mvtec-ad/).

## **Ejemplo Tranfer Learning (Transferencia de Aprendizaje)**

**Importaciones necesarias**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


import os
import numpy as np
import matplotlib.pyplot as plt

**Cargamos los datos** desde dropbox (respáldenlos para futuros experimentos, puede que los borre eventualmente). Otra forma sería dejar alojados los datos en Google Drive y montar el disco para leerlos directo desde su drive, yaque la memoria de disco de Colab es limitada ( en este ejemplo es suficiente).


In [ ]:
!wget https://www.dropbox.com/s/av931fekqljr0jj/hazelnut_data.zip
# descomprimimos el archivo con las imágenes
!unzip -q hazelnut_data.zip
!rm hazelnut_data.zip # eliminamos el archivo comprimido para liberar memoria

**Preparamos los datos usando los generadores de Tensorflow para estructurarlos en un formato de dataset** que no implique cargar todos los archivos en la ram a la vez, si no que se llamen cuando se necesiten durante el entrenamiento. En este caso el tamaño de la imagen que definiremos para los datasets tiene relación con el que aceptará el modelo pre-entrenado que cargaremos, en esta práctica el MobileNetV2, disponible [aquí](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

In [ ]:
# Parámetros que aceptará nuestro modelo posterior y para el entrenamiento (a gusto del consumidor!)
IMG_WIDTH, IMG_HEIGHT = 224, 224 # Para llevar las imagenes al tamaño del modelo pre-entrenado que cargaremos, cambiarlo adecuadamente según sea la necesidad
BATCH_SIZE            = 32
SEED_TRAIN_VAL        = 1 # semilla generadora para los datasets, debe ser igual para todos

# Una forma con una función que no eliminarán, la anterior ImageDataGenerator la sacarán en versiones futuras
path_base = '/content/DATA' # directorio actual donde están las carpetas con imágenes
train_ds = tf.keras.utils.image_dataset_from_directory(
                                                        path_base,
                                                        validation_split = 0.3,
                                                        subset           = "training",
                                                        seed             = SEED_TRAIN_VAL,
                                                        image_size       = (IMG_HEIGHT, IMG_WIDTH),
                                                        batch_size       = BATCH_SIZE)


val_ds = tf.keras.utils.image_dataset_from_directory(
                                                     path_base,
                                                     validation_split= 0.3,
                                                     subset          = "validation",
                                                     seed            = SEED_TRAIN_VAL,
                                                     image_size      = (IMG_HEIGHT, IMG_WIDTH),
                                                     batch_size      = BATCH_SIZE)


# creamos el conjunto de test, dejamos 20% para test y 10% para validación, del 30% original
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds     = val_ds.take((2*val_batches) // 3)
val_ds      = val_ds.skip((2*val_batches) // 3)

#nombre de las clases
class_names = train_ds.class_names
print(class_names)

Para un mayor control, estructurar los datos en una carpeta de train, otra de test, y otra de validación, que contengan a su vez sub-carpetas con los archivos de imágenes correspondientes.

In [ ]:
# visualización de algunas de las imágenes
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Configuración de los conjuntos de dato para un mejor rendimiento al llamar las imágenes desde el disco duro.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Generamos capas de pre-procesamiento y así aumentar el dataset original con algunas operaciones:

In [ ]:
data_augmentation = keras.Sequential([  layers.Rescaling(1./255, offset = -1), #para dejar los valores de intensidad entre [-1,1], lo que espera nuestro modelo mobilnetv2 que usaremos
                                                                               #verificar siempre según el modelo base que se vaya a usar cual es el rango esperado.
                                        layers.RandomFlip("horizontal_and_vertical"),
                                        layers.RandomRotation(0.2),
                                        layers.RandomZoom(.1, .1)
                                     ])

Usaremos MobileNetV2, si se usa otro, tener cuidado con las dimensiones de la imagen de entrada. Este modelo actuará como un extractor de características de las imagenes para luego pasarselas al clasificador

In [ ]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT,3)# agregamos la dimensión 3 yaque el modelo acepta imágenes a color
# Cargamos un modelo pre-entrenado en el dataset de ImageNet
modelo_base = keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

# es importante congelar la etapa convolucional para que no se entrene nuevamente
modelo_base.trainable = False

In [ ]:
N_EPOCAS  = 200 #máximo número de épocas

# Definición del modelo completo
modelo    = keras.Sequential([ layers.Input(shape=IMG_SHAPE),
                               data_augmentation,
                               modelo_base,
                               layers.GlobalAveragePooling2D(),
                               layers.Dropout(.2),
                               layers.Dense(1,activation = 'sigmoid')
                             ])


# Configuración de estrategia de detención temprana
PATIENCE = 20
early    = tf.keras.callbacks.EarlyStopping( patience=PATIENCE, monitor="val_loss", restore_best_weights=True )

# Configuración del entrenamiento
base_learning_rate = 0.01#tasa de aprendizaje base, se puede experimentar con este valor de partida, pero al usar transfer learning, usar valores bajos
modelo.compile(optimizer= keras.optimizers.Adam(lr=base_learning_rate),
               loss     = keras.losses.BinaryCrossentropy(from_logits=True),
               metrics  = ['accuracy'])

modelo.summary()

**Entrenamiento**

In [ ]:
# Entrenamiento
history = modelo.fit(train_ds, validation_data=val_ds, epochs=N_EPOCAS,callbacks = early)

Algunas gráficas del entrenamiento:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Accuracy en Entrenamiento')
plt.plot(val_acc, label='Accuracy en Validación')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Accuracy en Entrenamiento y Validación')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Pérdida en Entrenamiento')
plt.plot(val_loss, label='Pérdida en Validación')
plt.legend(loc='upper right')
plt.ylabel('Entropía Cruzada')
plt.ylim([0,1.0])
plt.title('Pérdida en Entrenamiento y Validación')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Resultados finales en training
print('Loss final en training: ',history.history['loss'][-1])
print('Accuracy final en training: ',history.history['accuracy'][-1])

**Resultados en conjunto de prueba**

In [ ]:
# Evaluamos en conjunto de prueba
score = modelo.evaluate(test_ds, verbose=0)

print('Loss en dataset de prueba: ',score[0])
print('Accuracy en dataset de prueba: ',score[1])

**Matriz de confusión en el conjunto de prueba**

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import seaborn as sns

y_pred = modelo.predict(test_ds, verbose = 0)#valores predichos con el modelo
y_pred = (y_pred>0.5).astype('int32') #binarización de la salida, yaque la sigmoidal entrega valores entre 0 y 1 a la salida de la red.

#etiquetas reales conjunto de test
y_test = np.concatenate([y for x, y in test_ds], axis=0)


cm   = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.show()

### **Resumiendo**
En esta práctica vimos:
- Cómo crear un dataset a partir de imágenes propias
- Reutilizar un modelo pre-entrenado en un dataset grande
- Implementar la estrategia de transferencia de aprendizaje
- Evaluar el desempeño de un modelo


## **Ejercicios Extra**

**E.1.** Repetir el proceso anterior pero con otros modelos pre-entrenados disponibles en tensorflow, comparar los resultados y quedarse con el mejor de acuerdo a los criterios que estime convenientes.

In [ ]:
#Código aquí

**E.2.** Repetir **E.1.** pero esta vez para **resolver un problema de clasificación multiclase** con imágenes r**ecolectadas por ustedes mismos**. Estructurar adecuadamente las carpetas con los datos, generar los datasets, definir los modelos, evaluar el desempeño del modelo y guardar el modelo para futuros usos.

In [ ]:
#Código aquí

**E.3.** Vimos que colab es una buena herramienta para entrenar modelos de deep learning yaque nos facilita el uso de GPUs de los servidores de Google. Generar un script .py de forma local, usando por ejemplo Spyder, para realizar inferencias en nuevas imágenes obtenidas en el punto **E.2.** y que utilice el modelo entrenado en colab. Los desafíos acá serán configurar adecuadamente un ambiente de python para trabajar con las librerías actualizadas o que requieran instalarse, con Anaconda esto se puede realizar de forma sencilla.

##**Referencias**

* Transfer Learning, CS231n: Convolutional Neural Networks for Visual Recognition, Curso Standford, 2022, [online](http://cs231n.stanford.edu/)
* Ejemplo de clasificación multiclase usando Transfer Learning: [video](https://www.youtube.com/watch?v=EkAg51oIvQI)
